In [1]:
ls

drive/  sample_data/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/colab/slt/

/content/drive/MyDrive/colab/slt


In [22]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=28822e292e42068bc2eb503a824d6faab5f367c7f32cfb862e86a64cf03d8345
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [6]:

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)

from transformers.models.mbart.modeling_mbart import MBartModel, MBartForConditionalGeneration
from transformers import MBartConfig

from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version



In [7]:
!python utils/jsonize.py --mode train dev test --src en --tgt gloss.asl

In [8]:
# f1 = open("./data/aslg.train.en", 'r', encoding = 'utf-8-sig').readlines()
# f2 = open("./data/aslg.train_processed.gloss.asl", 'r', encoding = 'utf-8-sig').readlines()

# with open("./data/text_dump.txt", 'w') as f:
#   f.writelines(f1+f2)
#   f.close()

In [9]:

logger = logging.getLogger(__name__)

# A list of all multilingual tokenizer which require src_lang and tgt_lang attributes.
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast, M2M100Tokenizer]


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    source_lang: str = field(default=None, metadata={"help": "Source language id for translation."})
    target_lang: str = field(default=None, metadata={"help": "Target language id for translation."})
    vocab_size: int = field(default=None, metadata={"help": "vocab size for training custom tokenizer"})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a jsonlines)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (sacrebleu) on a jsonlines file."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to evaluate the metrics (sacrebleu) on a jsonlines file."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": (
                "The maximum total sequence length for target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "The maximum total sequence length for validation target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
                "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
                "during ``evaluate`` and ``predict``."
            )
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to pad all samples to model maximum sentence length. "
                "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
                "efficient on GPU but very bad for TPU."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
                "which is used during ``evaluate`` and ``predict``."
            )
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default=None, metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )
    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The token to force as the first generated token after the :obj:`decoder_start_token_id`.Useful for"
                " multilingual models like :doc:`mBART <../model_doc/mbart>` where the first generated token needs to"
                " be the target language token.(Usually it is the target language token)"
            )
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        elif self.source_lang is None or self.target_lang is None:
            raise ValueError("Need to specify the source language and the target language.")

        # accepting both json and jsonl file extensions, as
        # many jsonlines files actually have a .json extension
        valid_extensions = ["json", "jsonl"]

        if self.train_file is not None:
            extension = self.train_file.split(".")[-1]
            assert extension in valid_extensions, "`train_file` should be a jsonlines file."
        if self.validation_file is not None:
            extension = self.validation_file.split(".")[-1]
            assert extension in valid_extensions, "`validation_file` should be a jsonlines file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))


In [10]:

model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--train_file", "./data/train_data.json",
    "--validation_file", "./data/dev_data.json",
    "--test_file", "./data/test_data.json",
    "--model_name_or_path", "lol", 
    "--output_dir" , "/output",
    "--source_lang", "en",
    "--target_lang", "gloss.asl",
    "--vocab_size", "3000",
    "--max_source_length", "50",
    "--max_target_length", "50",
    "--ignore_pad_token_for_loss", "True",
    "--output_dir","./result",
    "--generation_max_length", "50",
    "--generation_num_beams", "1",
    "--predict_with_generate", "True",
    "--per_device_train_batch_size", "64",
    "--per_device_eval_batch_size", "64",
    "--num_train_epochs", "1",
    "--learning_rate", "1e-5",
    "--do_train", "True",
    "--do_eval", "True",
    "--do_predict", "True",
    "--save_strategy", "no"
    ])

In [11]:
extension = "json"
raw_train_dataset = load_dataset( extension, data_files = data_args.train_file, split = "train" )
raw_validation_dataset = load_dataset( extension, data_files = data_args.validation_file, split = "train" )
raw_test_dataset = load_dataset( extension, data_files = data_args.test_file, split = "train" )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a3f577286ca63c9/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3659af2f1120ae79/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-af92e221f569b2c4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


In [12]:
def get_training_corpus():
    return (
        [ p for k in raw_train_dataset[i: i+1000]["translation"] for p in list(k.values()) ]
        for i in range(0, len(raw_train_dataset), 1000)
    )
corpus = get_training_corpus()

# train custom tokenizer 

In [13]:
# from tokenizers import ByteLevelBPETokenizer

# special_tokens = ['<s>', '</s>', 'pad', 'mask', 'unk', 'src', 'tgt']
# VOCAB_SIZE = 3000

# tokenizer = tokenizer = ByteLevelBPETokenizer()

# files = ['./data/text_dump.txt']
# # Customize training
# tokenizer.train(files=files, vocab_size=VOCAB_SIZE, min_frequency=2, special_tokens= special_tokens)


In [14]:
from transformers import MBartTokenizer
old_tokenizer = MBartTokenizerFast.from_pretrained("facebook/mbart-large-50")
tokenizer = old_tokenizer.train_new_from_iterator(corpus, data_args.vocab_size)

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
# testing the tokenizer

tgt_text = raw_train_dataset[0]["translation"]["gloss.asl"]
print(tgt_text)
labels = tokenizer(tgt_text, max_length = 10, padding = "max_length", return_tensors="pt").input_ids
print(tokenizer.convert_ids_to_tokens(list(labels[0])))


membership parliament see minute
['▁membership', '▁parliament', '▁see', '▁minute', '</s>', 'en_XX', '<pad>', '<pad>', '<pad>', '<pad>']


In [16]:
padding = False
prefix = data_args.source_prefix if data_args.source_prefix is not None else ""

def preprocess_function(examples):
    inputs = [ex[data_args.source_lang] for ex in examples["translation"]]
    targets = [ex[data_args.target_lang] for ex in examples["translation"]]
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(targets, max_length=data_args.max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

column_names = raw_train_dataset.column_names

In [17]:
train_dataset = raw_train_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

eval_dataset = raw_validation_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )
test_dataset = raw_test_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

Running tokenizer on train dataset:   0%|          | 0/83 [00:00<?, ?ba/s]

Running tokenizer on train dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on train dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
ids = list(train_dataset[0]["input_ids"])
print(tokenizer.convert_ids_to_tokens(ids))

label_ids = list(train_dataset[0]["labels"])
print(tokenizer.convert_ids_to_tokens(label_ids))
train_dataset[0]


['▁membership', '▁of', '▁parliament', '▁see', '▁minute', 's', '</s>', 'en_XX']
['▁membership', '▁parliament', '▁see', '▁minute', '</s>', 'en_XX']


{'input_ids': [1012, 72, 123, 118, 131, 59, 2, 7],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [1012, 123, 118, 131, 2, 7]}

In [19]:
set_seed(training_args.seed)

EMBEDDING_DIM = 512
SCALE_DOWN_FACTOR = 4

config = MBartConfig()

config.d_model = EMBEDDING_DIM
config.vocab_size = data_args.vocab_size
config.encoder_attention_heads //= SCALE_DOWN_FACTOR
config.encoder_ffn_dim //= SCALE_DOWN_FACTOR
config.encoder_layers //= SCALE_DOWN_FACTOR
config.decoder_attention_heads //= SCALE_DOWN_FACTOR
config.decoder_ffn_dim //= SCALE_DOWN_FACTOR
config.decoder_layers //= SCALE_DOWN_FACTOR

print(config)
model = MBartForConditionalGeneration(config)
model.resize_token_embeddings(len(tokenizer))


MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 4,
  "decoder_ffn_dim": 1024,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 3,
  "dropout": 0.1,
  "encoder_attention_heads": 4,
  "encoder_ffn_dim": 1024,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 3,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1024,
  "model_type": "mbart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": false,
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 3000
}



Embedding(3000, 512, padding_idx=1)

In [20]:
label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if training_args.fp16 else None,
)


In [23]:
# load metrics

# evaluate.list_evaluation_modules()

sacrebleu = evaluate.load("sacrebleu")
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = {}
    
    result["sacrebleu"] = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result["bleu_1"] = bleu.compute(predictions=decoded_preds, references=decoded_labels, max_order = 1)
    result["bleu_2"] = bleu.compute(predictions=decoded_preds, references=decoded_labels, max_order = 2)
    result["bleu_3"] = bleu.compute(predictions=decoded_preds, references=decoded_labels, max_order = 3)
    result["bleu_4"] = bleu.compute(predictions=decoded_preds, references=decoded_labels, max_order = 4)

    result["rouge"] = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    # result = {k: round(v, 4) for k, v in result.items()}
    return result



In [24]:
training_args.max_steps = -1
training_args.num_train_epochs = 50


# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if training_args.predict_with_generate else None,
)


In [25]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 82710
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 64650
You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,5.834900
1000,4.591900
1500,4.047200
2000,3.594500
2500,3.220000
3000,2.874400
3500,2.585200
4000,2.311500
4500,2.067100
5000,1.856800




Training completed. Do not forget to share your model on huggingface.co/models =)




In [151]:
print(train_result)

TrainOutput(global_step=30, training_loss=7.921477762858073, metrics={'train_runtime': 164.9146, 'train_samples_per_second': 11.642, 'train_steps_per_second': 0.182, 'total_flos': 7439083634688.0, 'train_loss': 7.921477762858073, 'epoch': 0.02})


In [26]:
trainer.save_model()  
trainer.save_metrics("train", train_result.metrics)


Saving model checkpoint to ./result
Configuration saved in ./result/config.json
Model weights saved in ./result/pytorch_model.bin
tokenizer config file saved in ./result/tokenizer_config.json
Special tokens file saved in ./result/special_tokens_map.json


In [27]:
predict_results = trainer.predict(
            test_dataset, metric_key_prefix="predict", max_length=training_args.generation_max_length, num_beams=training_args.generation_num_beams
        )

trainer.save_metrics("predict", predict_results.metrics)

if training_args.predict_with_generate:
    predictions = tokenizer.batch_decode(
        predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    predictions = [pred.strip() for pred in predictions]
    output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
    with open(output_prediction_file, "w", encoding="utf-8") as writer:
        writer.write("\n".join(predictions))


***** Running Prediction *****
  Num examples = 1000
  Batch size = 64


In [28]:
print(predict_results)

# for 

PredictionOutput(predictions=array([[  0,  82,  62, ...,   1,   1,   1],
       [  0,   0, 424, ...,   1,   1,   1],
       [  0,  79, 107, ...,   1,   1,   1],
       ...,
       [  0,   0,  63, ...,   1,   1,   1],
       [  0,   0,  62, ...,   1,   1,   1],
       [  0,  70,  62, ...,   1,   1,   1]]), label_ids=array([[1055,   62,   82, ..., -100, -100, -100],
       [ 283,  424,   75, ..., -100, -100, -100],
       [  79,  107,  399, ..., -100, -100, -100],
       ...,
       [1751,   63,  700, ...,    1,    1,    1],
       [  70,   62,  682, ...,    1,    1,    1],
       [  70,   62,  289, ...,    1,    1,    1]]), metrics={'predict_loss': 0.05588153749704361, 'predict_sacrebleu': {'score': 70.19756305499223, 'counts': [9898, 8095, 6457, 5045], 'totals': [10806, 9807, 8819, 7835], 'precisions': [91.59726078104757, 82.54308147241765, 73.21691801791586, 64.39055520102106], 'bp': 0.90849562706379, 'sys_len': 10806, 'ref_len': 11843}, 'predict_bleu_1': {'bleu': 0.832157108706033, '